# Works by John Milton

If the SPARQLWrapper has already been installed, please place a 'hash' in front of the command below. 

In [ ]:
!pip install SPARQLWrapper

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.core.display import display, HTML
import pandas as pd
import os
from collections import Counter
import plotly.express as px


import re
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

    
endpoint = 'https://data.bibliotheken.nl/sparql'
sparql = SPARQLWrapper(endpoint)

def run_query(sparql_query):
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    data = sparql.query().convert()
    df = pd.json_normalize(data['results']['bindings']) 
    return df


def count_items(identifier,name):
    items = Counter()

    item_names = dict()
    for i,row in df.iterrows():
        if item_names.get(row[identifier]) is None:
            item_names[row[identifier]] = row[name]

    for book in df['book.value'].unique():
        book_df = df[ df['book.value'] == book ]
        book_items = book_df[identifier].unique()
        for item in book_items:
            items.update([item_names[item]])
            
    return items

In the interface of the STCN, we can find that the ID for John Milton is [068421443](https://data.cerl.org/stcn_persons/068421443).

This identifier can be used to request all the works written by John Milton in the STCN. This identifier need to be appended to the base url http://data.bibliotheken.nl/id/thes/, and following the prefix 'p', as follows: 



[&lt;http://data.bibliotheken.nl/id/thes/p068421443&gt;](http://data.bibliotheken.nl/id/thes/p068421443)

In [ ]:
query = '''
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX kb: <http://data.bibliotheken.nl/def#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT * WHERE {

VALUES ?author_property { schema:author schema:contributor } 

?book schema:mainEntityOfPage ?mainEntity .
?mainEntity schema:isPartOf <http://data.bibliotheken.nl/id/dataset/stcn>  . 

?book schema:author|schema:contributor ?author_node . 
?author_node schema:author|schema:contributor <http://data.bibliotheken.nl/id/thes/p068421443> .

?book schema:name ?title . 

?book schema:publication ?publ .

OPTIONAL {
?publ schema:startDate ?publ_year . 
}

} 
'''

#schema:contributor

df = run_query(query)


In [ ]:
print(f'{df.shape[0]} books in total.\n\n')

for i,book in df.iterrows():
    print(f'{book["book.value"]}')
    print(f"{book['title.value']}\n{book['publ_year.value']}\n")

In [ ]:
years = Counter()
invalid = []

for i,row in df.sort_values( by = 'publ_year.value').iterrows():
    #print(row['publ_year.value'])
    if re.search(r'\d{4}',str(row['publ_year.value'])):
        years.update([int(row['publ_year.value'])])
    else:
        invalid.append(row['publ_year.value'])

if len(invalid)>0:
    print("Invalid years:")
    print(','.join(set(invalid)) )

x_axis = list(years.keys())
y_axis = list(years.values())

for year in range(min(x_axis),max(x_axis)):
    if year not in x_axis:
        x_axis.append(year)
        y_axis.append(0)
        

        
timeline = pd.DataFrame( {'year':x_axis,'number_of_titles':y_axis} )
timeline = timeline.sort_values(by='year')    


fig = px.line(timeline,
              x='year', 
              y='number_of_titles',
              width=600, 
              height=400)
fig.show()


## Publishers

In [ ]:
query = '''
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX kb: <http://data.bibliotheken.nl/def#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT * WHERE {
?book schema:mainEntityOfPage ?mainEntity .
?mainEntity schema:isPartOf <http://data.bibliotheken.nl/id/dataset/stcn>  . 

?book schema:author|schema:contributor ?author_node . 
?author_node schema:author|schema:contributor <http://data.bibliotheken.nl/id/thes/p068421443> .

?book schema:name ?title . 

?book schema:publication ?publ .

OPTIONAL {
?publ schema:publishedBy ?publisher . 
?publisher schema:name ?publ_name .
?publisher schema:location ?location_node .
?location_node schema:address ?address .
?address schema:addressLocality ?place . 
}

} 
'''

df = run_query(query)



In [ ]:
publishers = count_items('publisher.value','publ_name.value') 
        
x_axis = []
y_axis = []

for publisher,count in publishers.most_common():
    if not(pd.isna(publisher)) and not(re.search(r's[.]n[.]',publisher)):
        x_axis.append(publisher)
        y_axis.append(count)
        
x_label = 'Publisher'
y_label = 'Number of titles'
        
df_barchart = pd.DataFrame({x_label:x_axis,y_label:y_axis})


fig = px.bar(df_barchart , 
             y = x_label , 
             x= y_label,
            height = 1500)

fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

## Languages

In [ ]:
query = '''
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX kb: <http://data.bibliotheken.nl/def#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT * WHERE {
?book schema:mainEntityOfPage ?mainEntity .
?mainEntity schema:isPartOf <http://data.bibliotheken.nl/id/dataset/stcn>  . 

?book schema:author|schema:contributor ?author_node . 
?author_node schema:author|schema:contributor <http://data.bibliotheken.nl/id/thes/p068421443> .

?book schema:name ?title . 

?book schema:inLanguage ?language .

} 
'''

df = run_query(query)



In [ ]:
languages = Counter(df['language.value'])
        
x_axis = []
y_axis = []

for language, count in languages.most_common():
    x_axis.append(language)
    y_axis.append(count)


fig = px.bar(y = x_axis , 
             x= y_axis, 
             labels = {'x':'Number','y':'Language'}
            )

fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

## Formats

In [ ]:
query = '''
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX kb: <http://data.bibliotheken.nl/def#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT * WHERE {
?book schema:mainEntityOfPage ?mainEntity .
?mainEntity schema:isPartOf <http://data.bibliotheken.nl/id/dataset/stcn>  . 

?book schema:author|schema:contributor ?author_node . 
?author_node schema:author|schema:contributor <http://data.bibliotheken.nl/id/thes/p068421443> .

?book kb:bibliographicFormat ?format .
?book schema:inLanguage ?language .

} 
'''

df = run_query(query)




In [ ]:
formats = Counter(df['format.value'])
        
x_axis = []
y_axis = []

for bibl_format, count in formats.most_common():
    x_axis.append(bibl_format)
    y_axis.append(count)


fig = px.bar(y = x_axis , 
             x= y_axis, 
             labels = {'x':'Number','y':'Format'}
            )

fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

## Social Network Analysis

In [ ]:
query = '''

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX kb: <http://data.bibliotheken.nl/def#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT * {

?book schema:mainEntityOfPage ?mainEntity .
?mainEntity schema:isPartOf <http://data.bibliotheken.nl/id/dataset/stcn>  . 

?book schema:author|schema:contributor ?author_node . 
?author_node schema:author|schema:contributor <http://data.bibliotheken.nl/id/thes/p068421443> .


OPTIONAL {
?book schema:about ?subject .
?subject skos:prefLabel ?subject_label .
}

?book schema:publication ?publ .

OPTIONAL {
?publ schema:publishedBy ?publisher . 
?publisher schema:name ?publ_name .
}

}

'''
df = run_query(query)

In [ ]:
unique_ids = df['book.value'].unique().tolist()
edges = []

names = dict()
for i,row in df.iterrows():
    subject_id = row['subject.value']
    if not(pd.isna(subject_id)):
        subject_id = os.path.basename(subject_id)
        subject_name = row['subject_label.value']
        subject_name = re.sub(r',','',subject_name)
        names[subject_id]=(subject_name,'Subject')
    publ_id = row['publisher.value']
    if not(pd.isna(publ_id)):
        publ_id = os.path.basename(publ_id)
        publ_name = row['publ_name.value']
        publ_name = re.sub(r',','',publ_name)
        names[publ_id]=(publ_name,'Publisher')
    
for book in unique_ids:
    book_df = df[ df['book.value'] == book ]
    book_subjects = book_df['subject.value'].unique().tolist()
    book_publishers = book_df['publisher.value'].unique().tolist()

    for subject in book_subjects:
        for publisher in book_publishers:
            if not(pd.isna(subject)) and not(pd.isna(publisher)):
                edges.append( (os.path.basename(subject),os.path.basename(publisher)))

out = open("nodes.csv",'w',encoding='utf-8')
out.write('Id,Label,Type\n')
for name in names:
    out.write(f'{name},"{names[name][0]}","{names[name][1]}"\n')
out.close()

out = open("edges.csv",'w',encoding='utf-8')
out.write('Source,Target,Type\n')
for edge in edges:
    if not(pd.isna(edge[0])) and not(pd.isna(edge[1])): 
        out.write(f'{edge[0]},{edge[1]},"Undirected"\n')
out.close()